In [1]:
import sys
sys.path.append('../')
import pickle
from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.imdb_datareader import PopularityReader
from preprocessing.factorization_machine_transformer import  FactorizationMachineTransformer

user_item  = pickle.load(open("data/user_item.p", "rb")) 
users = pickle.load(open("data/users.p", "rb")) 
items = pickle.load(open("data/items.p", "rb")) 
train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]
popreader = PopularityReader()
pop_info = popreader.read_item_data(train_user_item)
for k, v in items.items():
    if k in pop_info:
        v.append(pop_info[k])
    else:
        v.append(0)


transformer = FactorizationMachineTransformer(users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(users, items, test_user_item)

In [2]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2


An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/fm-movielens'
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')
output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)


import io,boto3
import sagemaker.amazon.common as smac


def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
 
    
train_data = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    

In [5]:
print('Output: {}'.format(output_prefix))
print('Train data: {}'.format(train_data))

Output: s3://recommendation-demo-yianc/sagemaker/fm-movielens/output
Train data: s3://recommendation-demo-yianc/sagemaker/fm-movielens/train/train.protobuf


In [6]:
import sagemaker 

from sagemaker import get_execution_role
import boto3 
from sagemaker.amazon.amazon_estimator import get_image_uri
region = boto3.Session().region_name
container = get_image_uri(region, 'factorization-machines', 'latest')
container


'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


'382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest'

In [13]:

import boto3 
import sagemaker
from sagemaker import get_execution_role

crole = get_execution_role() 

dir(sagemaker.estimator.Estimator)
fm = sagemaker.estimator.Estimator(container,
                                   crole, 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())



fm.set_hyperparameters(
                      feature_dim=nFeatures,
                      predictor_type='binary_classifier',
#                       predictor_type='regressor',
                      mini_batch_size=200,
                      num_factors=64,
                      epochs=50)

fm.fit({'train': train_data})


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-18 12:45:15 Starting - Starting the training job...
2020-08-18 12:45:17 Starting - Launching requested ML instances.........
2020-08-18 12:46:58 Starting - Preparing the instances for training...
2020-08-18 12:47:44 Downloading - Downloading input data...
2020-08-18 12:48:17 Training - Downloading the training image...
2020-08-18 12:48:37 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[08/18/2020 12:48:39 INFO 140553220745024] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'

[2020-08-18 12:48:52.731] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 20, "duration": 1232, "num_examples": 400, "num_bytes": 8461936}
[08/18/2020 12:48:52 INFO 140553220745024] #quality_metric: host=algo-1, epoch=9, train binary_classification_accuracy <score>=0.6926
[08/18/2020 12:48:52 INFO 140553220745024] #quality_metric: host=algo-1, epoch=9, train binary_classification_cross_entropy <loss>=0.595904139519
[08/18/2020 12:48:52 INFO 140553220745024] #quality_metric: host=algo-1, epoch=9, train binary_f_1.000 <score>=0.727892360804
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1234.771966934204, "sum": 1234.771966934204, "min": 1234.771966934204}}, "EndTime": 1597754932.73156, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597754931.496322}

[08/18/2020 12:48:52 INFO 140553220745024] #progress_metric: host=algo-1, completed 20 % of epochs
#metrics {"Metrics": {"Max Ba

[2020-08-18 12:49:04.742] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 40, "duration": 1209, "num_examples": 400, "num_bytes": 8461936}
[08/18/2020 12:49:04 INFO 140553220745024] #quality_metric: host=algo-1, epoch=19, train binary_classification_accuracy <score>=0.6762125
[08/18/2020 12:49:04 INFO 140553220745024] #quality_metric: host=algo-1, epoch=19, train binary_classification_cross_entropy <loss>=0.640806147575
[08/18/2020 12:49:04 INFO 140553220745024] #quality_metric: host=algo-1, epoch=19, train binary_f_1.000 <score>=0.710532491479
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1211.0819816589355, "sum": 1211.0819816589355, "min": 1211.0819816589355}}, "EndTime": 1597754944.743542, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597754943.532015}

[08/18/2020 12:49:04 INFO 140553220745024] #progress_metric: host=algo-1, completed 40 % of epochs
#metrics {"Metrics"

[2020-08-18 12:49:16.825] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 60, "duration": 1189, "num_examples": 400, "num_bytes": 8461936}
[08/18/2020 12:49:16 INFO 140553220745024] #quality_metric: host=algo-1, epoch=29, train binary_classification_accuracy <score>=0.6878125
[08/18/2020 12:49:16 INFO 140553220745024] #quality_metric: host=algo-1, epoch=29, train binary_classification_cross_entropy <loss>=0.636066596723
[08/18/2020 12:49:16 INFO 140553220745024] #quality_metric: host=algo-1, epoch=29, train binary_f_1.000 <score>=0.720277762222
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1191.831111907959, "sum": 1191.831111907959, "min": 1191.831111907959}}, "EndTime": 1597754956.826019, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597754955.633724}

[08/18/2020 12:49:16 INFO 140553220745024] #progress_metric: host=algo-1, completed 60 % of epochs
#metrics {"Metrics": {

[2020-08-18 12:49:22.797] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 70, "duration": 1177, "num_examples": 400, "num_bytes": 8461936}
[08/18/2020 12:49:22 INFO 140553220745024] #quality_metric: host=algo-1, epoch=34, train binary_classification_accuracy <score>=0.696275
[08/18/2020 12:49:22 INFO 140553220745024] #quality_metric: host=algo-1, epoch=34, train binary_classification_cross_entropy <loss>=0.631325893402
[08/18/2020 12:49:22 INFO 140553220745024] #quality_metric: host=algo-1, epoch=34, train binary_f_1.000 <score>=0.727607004327
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1179.6588897705078, "sum": 1179.6588897705078, "min": 1179.6588897705078}}, "EndTime": 1597754962.797828, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597754961.617797}

[08/18/2020 12:49:22 INFO 140553220745024] #progress_metric: host=algo-1, completed 70 % of epochs
#metrics {"Metrics":


2020-08-18 12:49:44 Uploading - Uploading generated training model[2020-08-18 12:49:34.857] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 90, "duration": 1224, "num_examples": 400, "num_bytes": 8461936}
[08/18/2020 12:49:34 INFO 140553220745024] #quality_metric: host=algo-1, epoch=44, train binary_classification_accuracy <score>=0.6547
[08/18/2020 12:49:34 INFO 140553220745024] #quality_metric: host=algo-1, epoch=44, train binary_classification_cross_entropy <loss>=0.757938801193
[08/18/2020 12:49:34 INFO 140553220745024] #quality_metric: host=algo-1, epoch=44, train binary_f_1.000 <score>=0.688181510328
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1226.3479232788086, "sum": 1226.3479232788086, "min": 1226.3479232788086}}, "EndTime": 1597754974.857643, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597754973.630927}

[08/18/2020 12:49:34 INFO 140553220745024] #progress_m


2020-08-18 12:49:51 Completed - Training job completed
Training seconds: 127
Billable seconds: 127


In [14]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

In [15]:
sys.path.append('../')
import sagemaker_utils
from sagemaker_utils.query_serializer import serialize as fmserialize 
from sagemaker.predictor import  json_deserializer
from sklearn.metrics import accuracy_score
import numpy 


sagemaker_utils.query_serializer.nFeatures = nFeatures
fm_predictor.content_type = sagemaker_utils.query_serializer.CONTENT_TYPE
fm_predictor.serializer = fmserialize
fm_predictor.deserializer = json_deserializer


def model_accuracy(X_test, Y_test): 
    X_test_arr = X_test
        
    result = fm_predictor.predict(X_test_arr) 
    y_pred = [] 
    for p in result['predictions']: 
        if p['score'] > 0.5:
            y_pred.append(1)
        else: 
            y_pred.append(0)
    return accuracy_score(Y_test, y_pred, normalize=False)

In [16]:
accuracy = model_accuracy(X_test, Y_test) / len(Y_test)
accuracy 

0.6915822563744324

In [17]:
accuracy = model_accuracy(X_cold_test, Y_cold_test) / len(Y_cold_test)
accuracy

0.6853932584269663